<a href="https://colab.research.google.com/github/interactiveaudiolab/MSG/blob/main/MSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make it Sound Good (MSG) Music Source Separation Post-Processor

Colab demo for the 2022 ISMIR paper **Music Separation Enhancement with Generative Modeling**.

[Click here](https://ismir2022program.ismir.net/poster_225.html) to see the abstract, paper, poster, and video.

If you use this notebook for research purposes, we kindly ask that you use the following bibtex entry to cite this work:

```
@inproceedings{schaffer2022music,
  title={Music Separation Enhancement with Generative Modeling},
  author={Schaffer, Noah and Cogan, Boaz and Manilow, Ethan and Morrison, Max and Seetharaman, Prem and Pardo, Bryan},
  booktitle={International Society for Music Information Retrieval (ISMIR)},
  month={December},
  year={2022}
}
```

In [ ]:
#@title Install requirements (run once before using)

%%capture
!pip install git+https://github.com/nussl/nussl.git@salient_mixsrc2
!git clone https://github.com/interactiveaudiolab/MSG.git
!pip install -r /content/MSG/requirements.txt
!apt-get install sox

In [ ]:
#@title Download models (run once before using)

import os
import urllib.request
from pathlib import Path, PurePath


def download_models(base_path: Path = '/content/msg_checkpoints'):
  """Downloads models from HuggingFace and saves them locally.

  Args:


  Returns:
    Dictionary where keys the model names, values are local checkpoint paths.
  """
  model_urls = {
      'bass': 'https://huggingface.co/boazcogan/MSG_pretrained_checkpoints/resolve/main/bass29netG.pt',
      'drums': 'https://huggingface.co/boazcogan/MSG_pretrained_checkpoints/resolve/main/drums29netG.pt',
      'vocals': 'https://huggingface.co/boazcogan/MSG_pretrained_checkpoints/resolve/main/vocals39netG.pt',
  }

  os.makedirs(base_path, exist_ok=True)

  model_paths = {}
  for i, (nm, url) in enumerate(model_urls.items()):
    model_fname = url.split('/')[-1]
    out_path = PurePath(base_path, model_fname)
    if os.path.isfile(out_path):
      print(f'Found {out_path}, skipping download...')
    else:
      print(f'Downloading {nm:6} model to '
            f'{str(out_path)+",":41} {i+1} of {len(model_urls)}.')
      urllib.request.urlretrieve(url, out_path)
    model_paths[nm] = out_path

  return model_paths

model_paths = download_models()


In [ ]:
#@title Run inference

from google.colab import files
from IPython.display import Audio, display
import soundfile as sf


uploaded = files.upload()
file_nm = list(uploaded.keys())[0]
in_file_path = PurePath('/content', file_nm)
display('Input file:')
display(Audio(filename=in_file_path))

model = 'vocals' #@param ["bass", "drums", "vocals"]
selected_model_path = model_paths[model]

display(f'Running MSG inference on {in_file_path}')
!python /content/MSG/Run_Inference.py -a $in_file_path -g $selected_model_path

out_file_path = PurePath('/content/msg_output', file_nm)

display(f'MSG processed file ({out_file_path}):')
display(Audio(filename=out_file_path))